# Stokes/Correlation Conversion for Visibility Data

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/demo_stokes_correlation_conversion.ipynb)

This notebook demonstrates how to convert visibility data between feed basis (correlation products) and Stokes basis using the `corr_to_stokes` and `stokes_to_corr` functions.

**Key Concepts:**
- **Correlation products** represent the raw output from interferometer feeds (e.g., XX, XY, YX, YY for linear feeds or RR, RL, LR, LL for circular feeds)
- **Stokes parameters** (I, Q, U, V) represent physical polarization properties.
  
**This tutorial covers:**
1. Linear polarization conversion (XX, XY, YX, YY ↔ I, Q, U, V)
2. Circular polarization conversion (RR, RL, LR, LL ↔ I, Q, U, V)
3. Round-trip conversion verification
4. Working with multidimensional visibility arrays
5. Working with xarray DataArrays (extract `.values` for low-level functions)
6. Higher-level image conversion functions (accept both numpy and xarray)

## Install AstroVIPER

Skip this cell if you don't want to install the latest version of AstroVIPER.

In [ ]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

## Imports

In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
from astroviper.core.imaging.imaging_utils.corr_to_stokes import corr_to_stokes, stokes_to_corr, image_stokes_to_corr, image_corr_to_stokes

# Set random seed for reproducibility
np.random.seed(42)

## 1. Linear Polarization: Correlation Products ↔ Stokes Parameters

Linear feeds produce correlation products in the order: **[XX, XY, YX, YY]**

These convert to Stokes parameters **[I, Q, U, V]** using:
- I = XX + YY
- Q = XX - YY
- U = XY + YX
- V = i(YX - XY)

### Generate synthetic linear correlation visibility data

We'll create a simple example with known properties to verify the conversion.

In [ ]:
# Create synthetic visibility data for a source with:
# - Total intensity I = 10.0 Jy
# - Linear polarization Q = 2.0 Jy (slightly more power in XX than YY)
# - No linear polarization at 45° (U = 0)
# - No circular polarization (V = 0)

# From the inverse formulas:
# XX = (I + Q)/2 = (10 + 2)/2 = 6.0
# XY = (U + iV)/2 = (0 + 0)/2 = 0.0
# YX = (U - iV)/2 = (0 - 0)/2 = 0.0
# YY = (I - Q)/2 = (10 - 2)/2 = 4.0


linear_corr = np.array([6.0 + 0.0j, 0.0 + 0.0j, 0.0 + 0.0j, 4.0 + 0.0j])

print("Input linear correlation products [XX, XY, YX, YY]:")
print(linear_corr)
print(f"\nShape: {linear_corr.shape}")

### Convert correlation products to Stokes parameters

In [ ]:
# Convert to Stokes parameters
stokes_linear = corr_to_stokes(linear_corr, corr_type='linear')

print("Output Stokes parameters [I, Q, U, V]:")
print(stokes_linear)
print(f"\nExpected: [10.0, 2.0, 0.0, 0.0]")
print(f"Match: {np.allclose(stokes_linear, [10.0, 2.0, 0.0, 0.0])}")

### Convert Stokes parameters back to correlation products

In [ ]:
# Convert back to correlation products
linear_corr_roundtrip = stokes_to_corr(stokes_linear, corr_type='linear')

print("Round-trip correlation products [XX, XY, YX, YY]:")
print(linear_corr_roundtrip)
print(f"\nOriginal:")
print(linear_corr)
print(f"\nRound-trip successful: {np.allclose(linear_corr, linear_corr_roundtrip)}")

## 2. Circular Polarization: Correlation Products ↔ Stokes Parameters

Circular feeds produce correlation products in the order: **[RR, RL, LR, LL]**

These convert to Stokes parameters **[I, Q, U, V]** using:
- I = RR + LL
- Q = RL + LR
- U = i(LR - RL)
- V = RR - LL

### Generate synthetic circular correlation visibility data

In [ ]:
# Create synthetic visibility data for a source with:
# - Total intensity I = 8.0 Jy
# - No linear polarization (Q = 0, U = 0)
# - Circular polarization V = 2.0 Jy (more right-hand than left-hand)

# From the inverse formulas:
# RR = (I + V)/2 = (8 + 2)/2 = 5.0
# RL = (Q + iU)/2 = (0 + 0)/2 = 0.0
# LR = (Q - iU)/2 = (0 - 0)/2 = 0.0
# LL = (I - V)/2 = (8 - 2)/2 = 3.0


circular_corr = np.array([5.0 + 0.0j, 0.0 + 0.0j, 0.0 + 0.0j, 3.0 + 0.0j])

print("Input circular correlation products [RR, RL, LR, LL]:")
print(circular_corr)
print(f"\nShape: {circular_corr.shape}")

### Convert correlation products to Stokes parameters

In [ ]:
# Convert to Stokes parameters
stokes_circular = corr_to_stokes(circular_corr, corr_type='circular')

print("Output Stokes parameters [I, Q, U, V]:")
print(stokes_circular)
print(f"\nExpected: [8.0, 0.0, 0.0, 2.0]")
print(f"Match: {np.allclose(stokes_circular, [8.0, 0.0, 0.0, 2.0])}")

### Convert Stokes parameters back to correlation products

In [ ]:
# Convert back to correlation products
circular_corr_roundtrip = stokes_to_corr(stokes_circular, corr_type='circular')

print("Round-trip correlation products [RR, RL, LR, LL]:")
print(circular_corr_roundtrip)
print(f"\nOriginal:")
print(circular_corr)
print(f"\nRound-trip successful: {np.allclose(circular_corr, circular_corr_roundtrip)}")

## 3. Multidimensional Visibility Arrays

Real visibility data has multiple dimensions: time, baseline, frequency, and polarization.
The low level conversion functions work on any shape as long as the polarization dimension is last.

In [ ]:
# Create realistic multi-dimensional visibility data
# Shape: (time=10, baseline=15, frequency=32, polarization=4)
n_time = 10
n_baseline = 15
n_freq = 32
n_pol = 4

# Generate random complex visibility data
# In reality, this would come from your measurement set
vis_corr = np.random.randn(n_time, n_baseline, n_freq, n_pol) + \
           1j * np.random.randn(n_time, n_baseline, n_freq, n_pol)

print(f"Input visibility shape: {vis_corr.shape}")
print(f"Dimensions: (time={n_time}, baseline={n_baseline}, frequency={n_freq}, polarization={n_pol})")

In [ ]:
# Convert to Stokes
vis_stokes = corr_to_stokes(vis_corr, corr_type='linear')

print(f"\nStokes visibility shape: {vis_stokes.shape}")
print("The shape is preserved, only the last dimension is transformed.")

In [ ]:
# Convert back to correlation products
vis_corr_roundtrip = stokes_to_corr(vis_stokes, corr_type='linear')

print(f"\nRound-trip visibility shape: {vis_corr_roundtrip.shape}")
print(f"Round-trip successful: {np.allclose(vis_corr, vis_corr_roundtrip)}")
print(f"Maximum absolute error: {np.max(np.abs(vis_corr - vis_corr_roundtrip))}")

## 4. Working with xarray DataArrays

The low-level conversion functions (`corr_to_stokes`, `stokes_to_corr`) accept **only numpy arrays**.
If you have an xarray DataArray, extract the underlying numpy array using `.values` before calling these functions.

For convenience with image data, use the higher-level functions (`image_corr_to_stokes`, `image_stokes_to_corr`) which accept both numpy arrays and xarray DataArrays directly.

In [ ]:
# Create an xarray DataArray with labeled dimensions and coordinates
vis_xarray = xr.DataArray(
    vis_corr,
    dims=['time', 'baseline_id', 'frequency', 'polarization'],
    coords={
        'time': np.arange(n_time),
        'baseline_id': np.arange(n_baseline),
        'frequency': np.linspace(1.4e9, 1.5e9, n_freq),  # 1.4-1.5 GHz
        'polarization': ['XX', 'XY', 'YX', 'YY']
    }
)

print("Input xarray DataArray:")
print(vis_xarray)
print(f"\nData type: {type(vis_xarray)}")

In [ ]:
# Extract numpy array from xarray using .values, then convert to Stokes
# Low-level functions only accept numpy arrays
vis_stokes_from_xarray = corr_to_stokes(vis_xarray.values, corr_type='linear')

print(f"Output type: {type(vis_stokes_from_xarray)}")
print(f"Output shape: {vis_stokes_from_xarray.shape}")
print("\nNote: Low-level functions accept only numpy arrays.")
print("Use .values to extract the underlying array from xarray DataArrays.")

In [ ]:
# Verify the xarray conversion matches the numpy conversion
print(f"\nxarray and numpy conversions match: {np.allclose(vis_stokes, vis_stokes_from_xarray)}")

## 5. Image Conversion with Higher-Level Functions

The low-level functions shown above (`corr_to_stokes` and `stokes_to_corr`) work on **numpy arrays** where **polarization is the last dimension**.

However, image data have a different structure:
- **Image data**: `(time, frequency, polarization, l, m)`

For image data, use the higher-level wrapper functions:
- `image_corr_to_stokes()` - converts image correlation products to Stokes
- `image_stokes_to_corr()` - converts image Stokes to correlation products

These functions:
- Handle non-last polarization dimension via the `pol_axis` parameter (default: `pol_axis=2`)
- **Accept both numpy arrays and xarray DataArrays** (unlike the low-level functions)
- Preserve xarray structure, dimensions, coordinates, and attributes
- Automatically update polarization coordinates for xarray output

### 5.1 Regular Image Structure (pol_axis=2)

Let's create a synthetic image cube with the typical `image_xds["SKY"]` structure where polarization is at axis 2.

In [ ]:
# Create synthetic image data
# Shape: (time=1, frequency=64, polarization=4, l=128, m=128)
# This matches the typical image_xds["SKY"] structure

n_time_img = 1
n_freq_img = 64
n_pol_img = 4
n_l = 128
n_m = 128

# Generate random complex image data (correlation products)
# Shape: (1, 64, 4, 128, 128) where polarization is at axis 2 (not last!)
image_corr_np = np.random.randn(n_time_img, n_freq_img, n_pol_img, n_l, n_m) + \
                1j * np.random.randn(n_time_img, n_freq_img, n_pol_img, n_l, n_m)

print(f"Input image correlation shape: {image_corr_np.shape}")
print(f"Dimensions: (time={n_time_img}, freq={n_freq_img}, pol={n_pol_img}, l={n_l}, m={n_m})")
print(f"pol_axis=2 is the default for image_corr_to_stokes()")

In [ ]:
# Convert correlation image to Stokes image
# No need to specify pol_axis since default is 2
image_stokes_np = image_corr_to_stokes(image_corr_np, corr_type='linear')

print(f"Output Stokes image shape: {image_stokes_np.shape}")
print("Shape is preserved, only the polarization dimension is transformed.")

In [ ]:
# Convert Stokes image back to correlation image (round-trip test)
image_corr_roundtrip_np = image_stokes_to_corr(image_stokes_np, corr_type='linear')

print(f"Round-trip image shape: {image_corr_roundtrip_np.shape}")
print(f"Round-trip successful: {np.allclose(image_corr_np, image_corr_roundtrip_np)}")
print(f"Maximum absolute error: {np.max(np.abs(image_corr_np - image_corr_roundtrip_np))}")

### 5.2 Image with Polarization at Last Axis (pol_axis=-1)

For images where polarization is the last dimension, you can use `pol_axis=-1`.

In [ ]:
# Create image with polarization at last axis
# Shape: (time=1, frequency=32, l=64, m=64, polarization=4)
n_time_img2 = 1
n_freq_img2 = 32
n_l2 = 64
n_m2 = 64
n_pol2 = 4

image_pol_last = np.random.randn(n_time_img2, n_freq_img2, n_l2, n_m2, n_pol2) + \
                 1j * np.random.randn(n_time_img2, n_freq_img2, n_l2, n_m2, n_pol2)

print(f"Input image shape (pol at last axis): {image_pol_last.shape}")
print(f"Dimensions: (time={n_time_img2}, freq={n_freq_img2}, l={n_l2}, m={n_m2}, pol={n_pol2})")

In [ ]:
# Convert with pol_axis=-1
stokes_pol_last = image_corr_to_stokes(image_pol_last, corr_type='linear', pol_axis=-1)
print(f"Stokes image shape: {stokes_pol_last.shape}")
print("Shape is preserved, only the last dimension is transformed.")

In [ ]:
# Round-trip test
corr_pol_last_rt = image_stokes_to_corr(stokes_pol_last, corr_type='linear', pol_axis=-1)
print(f"Round-trip image shape: {corr_pol_last_rt.shape}")
print(f"Round-trip successful: {np.allclose(image_pol_last, corr_pol_last_rt)}")
print(f"Maximum absolute error: {np.max(np.abs(image_pol_last - corr_pol_last_rt))}")

In [ ]:
# Note: When pol_axis=-1, image_corr_to_stokes produces the same result as corr_to_stokes
# (since polarization is already at the last axis)
stokes_lowlevel = corr_to_stokes(image_pol_last, corr_type='linear')
print(f"Equivalent to low-level corr_to_stokes: {np.allclose(stokes_pol_last, stokes_lowlevel)}")

### 5.3 xarray DataArray Examples

The higher-level functions preserve xarray structure, coordinates, and attributes when given xarray input.

In [ ]:
# Example with regular image structure (pol_axis=2)
image_corr_xr = xr.DataArray(
    image_corr_np,
    dims=['time', 'frequency', 'polarization', 'l', 'm'],
    coords={
        'time': [0.0],
        'frequency': np.linspace(1.4e9, 1.5e9, n_freq_img),  # 1.4-1.5 GHz
        'polarization': ['XX', 'XY', 'YX', 'YY'],
        'l': np.arange(n_l),
        'm': np.arange(n_m)
    },
    attrs={
        'telescope': 'VLA',
        'field': 'MySource',
        'units': 'Jy/pixel'
    }
)

print("Input xarray image (correlation products):")
print(image_corr_xr)
print(f"\nPolarization coordinates: {image_corr_xr.polarization.values}")

In [ ]:
# Convert to Stokes - output is also an xarray DataArray!
image_stokes_xr = image_corr_to_stokes(image_corr_xr, corr_type='linear')

print("Output xarray image (Stokes parameters):")
print(image_stokes_xr)
print(f"\nOutput type: {type(image_stokes_xr)}")
print(f"Polarization coordinates: {image_stokes_xr.polarization.values}")
print(f"\nNote: Polarization coordinates automatically updated to ['I', 'Q', 'U', 'V']")
print(f"Attributes preserved: {image_stokes_xr.attrs}")

In [ ]:
# Round-trip conversion
image_corr_roundtrip_xr = image_stokes_to_corr(image_stokes_xr, corr_type='linear')

print("Round-trip xarray image:")
print(f"Polarization coordinates: {image_corr_roundtrip_xr.polarization.values}")
print(f"Round-trip successful: {np.allclose(image_corr_xr.values, image_corr_roundtrip_xr.values)}")
print(f"Maximum absolute error: {np.max(np.abs(image_corr_xr.values - image_corr_roundtrip_xr.values))}")